In [1]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from itertools import combinations 

2022-08-12 13:23:44.839722: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-12 13:23:44.839751: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import PolynomialFeatures
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout, Input
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras

In [3]:
cols_1 = [
    'pot_ativa_inj_barramento11',
     'pot_ativa_inj_barramento12',
     'pot_ativa_inj_barramento24',  
      'pot_reativa_inj_barramento11',
     'pot_reativa_inj_barramento12',
     'pot_reativa_inj_barramento24',  
     'tensao_barramento11',
     'tensao_barramento12',
     'tensao_barramento24',
     'carga_subrede_138kv',
     'pot_eolica_subrede_138kv',
     'pot_solar_subrede_138kv'
]

In [4]:
def augment(x, y, augmenter = None):
    x, augmenter = _add_interactions(x, augmenter)
    return x, y, augmenter, x.columns

def _add_interactions(df, augmenter):
    combos = list(combinations(list(df.columns), 2))
    colnames = list(df.columns) + ['_'.join(x) for x in combos]

    if augmenter:
        df = augmenter.transform(df)
    else:
        augmenter = PolynomialFeatures(interaction_only=True, include_bias=False)
        df = augmenter.fit_transform(df)

    df = pd.DataFrame(df)
    df.columns = colnames
    
    noint_indicies = [i for i, x in enumerate(list((df == 0).all())) if x]
    df = df.drop(df.columns[noint_indicies], axis=1)
    
    return df, augmenter

In [108]:
def run(df, cols, cols_pred, model_id, update_test = False):
    _df = df[cols + cols_pred]
    
    X, Y = _df[cols], _df[cols_pred]
    X, Y, _, x_cols, = augment(X, Y)
    
    _df_augm = pd.concat([X, Y],  axis=1)
    
    test_split=round(len(_df_augm)*0.30)
    df_for_training=_df_augm[:-test_split]
    df_for_testing=_df_augm[-test_split:]
    
    scaler = StandardScaler()
    df_for_training_scaled = scaler.fit_transform(df_for_training)
    df_for_testing_scaled=scaler.transform(df_for_testing)
    
    trainX, trainY = df_for_training_scaled[:, :-len(cols_pred)], df_for_training_scaled[:, -len(cols_pred):]
    testX, testY = df_for_testing_scaled[:, :-len(cols_pred)], df_for_testing_scaled[:, -len(cols_pred):]
    
    print(f" ========== MODEL {model_id} ============")
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
    mc = ModelCheckpoint(f"models/ann-{model_id}.h5", monitor='val_loss', mode='min', save_best_only=True, verbose=1)
    
    model = Sequential()
    model.add(Input(shape=(trainX.shape[1],)))
    model.add(Dense(trainX.shape[-1] ** 3, activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(trainX.shape[-1] ** 2, activation = 'relu')) 
    model.add(Dropout(0.2))
    model.add(Dense(len(cols_pred)))
    
    model.compile(loss = 'mse', optimizer = 'adam')
    model.fit(
        trainX, trainY,
        batch_size = 10, epochs = 20,
        callbacks = [es, mc],
        validation_data = (testX, testY)
    )
    
    _predictions = model.predict(testX)
    
    predictions = inverse_transform(scaler, testX, _predictions)
    originals = inverse_transform(scaler, testX, testY)
    
    print("RMSE: " + str(sqrt(mean_squared_error(originals, predictions))))
    
    if update_test:
        df_for_testing[cols_pred[0]] = predictions[:,-1]
    
    return pd.concat([df_for_training, df_for_testing])
    
def inverse_transform(scaler, testX, arrs):
    zeros_padding = np.zeros(testX.shape[-1])
    arrs_copies = list(map(lambda arr: np.concatenate((zeros_padding, arr)), arrs))
    return scaler.inverse_transform(np.reshape(arrs_copies, (len(arrs), testX.shape[-1] + arrs.shape[-1])))

In [104]:
input_df = pd.read_csv("input_data_nn.csv", sep=";")
output_df = pd.read_csv("output_data_nn.csv", sep=";")

In [105]:
df = pd.concat([input_df, output_df], axis=1)
df = df.sample(frac=1).reset_index(drop=True)

In [106]:
cols = ['carga_subrede_138kv',
        'pot_eolica_subrede_138kv',
        'pot_solar_subrede_138kv']

cols_pred = ['tensao_barramento12']
model_id = 'tensao_12_mse'
tensao_12_df = run(df, cols, cols_pred, model_id, update_test=True)

cols = ['carga_subrede_138kv',
        'pot_eolica_subrede_138kv',
        'pot_solar_subrede_138kv']

cols_pred = ['tensao_barramento11']
model_id = 'tensao_11_mse'
tensao_11_df = run(df, cols, cols_pred, model_id, update_test=True)

cols = ['carga_subrede_138kv',
        'pot_eolica_subrede_138kv',
        'pot_solar_subrede_138kv']

cols_pred = ['tensao_barramento24']
model_id = 'tensao_24_mse'
tensao_24_df = run(df, cols, cols_pred, model_id, update_test=True)

df['tensao_barramento11'] = tensao_11_df['tensao_barramento11']
df['tensao_barramento12'] = tensao_12_df['tensao_barramento12']
df['tensao_barramento24'] = tensao_24_df['tensao_barramento24']

 ========== MODEL tensao_12_mse ============
Epoch 1/20
607/614 [============================>.] - ETA: 0s - loss: 0.7874
Epoch 1: val_loss improved from inf to 0.73337, saving model to models/ann-tensao_12_mse.h5
614/614 [==============================] - 2s 3ms/step - loss: 0.7866 - val_loss: 0.7334
Epoch 2/20
606/614 [============================>.] - ETA: 0s - loss: 0.7426
Epoch 2: val_loss improved from 0.73337 to 0.71851, saving model to models/ann-tensao_12_mse.h5
614/614 [==============================] - 2s 3ms/step - loss: 0.7414 - val_loss: 0.7185
Epoch 3/20
590/614 [===========================>..] - ETA: 0s - loss: 0.7258
Epoch 3: val_loss improved from 0.71851 to 0.70227, saving model to models/ann-tensao_12_mse.h5
614/614 [==============================] - 1s 2ms/step - loss: 0.7271 - val_loss: 0.7023
Epoch 4/20
608/614 [============================>.] - ETA: 0s - loss: 0.7118
Epoch 4: val_loss improved from 0.70227 to 0.69636, saving model to models/ann-tensao_12_mse.h5


/home/daniel/anaconda3/envs/ipynb_py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


605/614 [============================>.] - ETA: 0s - loss: 0.7536
Epoch 1: val_loss improved from inf to 0.69906, saving model to models/ann-tensao_11_mse.h5
614/614 [==============================] - 2s 3ms/step - loss: 0.7528 - val_loss: 0.6991
Epoch 2/20
602/614 [============================>.] - ETA: 0s - loss: 0.7221
Epoch 2: val_loss did not improve from 0.69906
614/614 [==============================] - 2s 3ms/step - loss: 0.7206 - val_loss: 0.6994
Epoch 3/20
595/614 [============================>.] - ETA: 0s - loss: 0.6981
Epoch 3: val_loss improved from 0.69906 to 0.67676, saving model to models/ann-tensao_11_mse.h5
614/614 [==============================] - 1s 2ms/step - loss: 0.7015 - val_loss: 0.6768
Epoch 4/20
609/614 [============================>.] - ETA: 0s - loss: 0.6893
Epoch 4: val_loss did not improve from 0.67676
614/614 [==============================] - 1s 2ms/step - loss: 0.6899 - val_loss: 0.6858
Epoch 5/20
602/614 [============================>.] - ETA: 0s - l

/home/daniel/anaconda3/envs/ipynb_py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


586/614 [===========================>..] - ETA: 0s - loss: 0.8025
Epoch 1: val_loss improved from inf to 0.78450, saving model to models/ann-tensao_24_mse.h5
614/614 [==============================] - 2s 2ms/step - loss: 0.8042 - val_loss: 0.7845
Epoch 2/20
600/614 [============================>.] - ETA: 0s - loss: 0.7688
Epoch 2: val_loss improved from 0.78450 to 0.77895, saving model to models/ann-tensao_24_mse.h5
614/614 [==============================] - 1s 2ms/step - loss: 0.7681 - val_loss: 0.7790
Epoch 3/20
610/614 [============================>.] - ETA: 0s - loss: 0.7492
Epoch 3: val_loss improved from 0.77895 to 0.75217, saving model to models/ann-tensao_24_mse.h5
614/614 [==============================] - 1s 2ms/step - loss: 0.7495 - val_loss: 0.7522
Epoch 4/20
602/614 [============================>.] - ETA: 0s - loss: 0.7381
Epoch 4: val_loss improved from 0.75217 to 0.74050, saving model to models/ann-tensao_24_mse.h5
614/614 [==============================] - 1s 2ms/step -

/home/daniel/anaconda3/envs/ipynb_py37/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [85]:
cols = ['carga_subrede_138kv',
        'pot_eolica_subrede_138kv',
        'pot_solar_subrede_138kv',
        'tensao_barramento11',
        'tensao_barramento12',
         'tensao_barramento24']

cols_pred = ['pot_ativa_inj_barramento11']
model_id = 'pot_ativa_inj_barramento11_mse'
pot_ativa_inj_barramento11_df = run(df, cols, cols_pred, model_id)

 ========== MODEL pot_ativa_inj_barramento11_mse ============
Epoch 1/20
614/614 [==============================] - ETA: 0s - loss: 0.0722
Epoch 1: val_loss improved from inf to 0.05805, saving model to models/ann-pot_ativa_inj_barramento11_mse.h5
614/614 [==============================] - 19s 31ms/step - loss: 0.0722 - val_loss: 0.0581
Epoch 2/20
613/614 [============================>.] - ETA: 0s - loss: 0.0428
Epoch 2: val_loss improved from 0.05805 to 0.05520, saving model to models/ann-pot_ativa_inj_barramento11_mse.h5
614/614 [==============================] - 20s 33ms/step - loss: 0.0428 - val_loss: 0.0552
Epoch 3/20
613/614 [============================>.] - ETA: 0s - loss: 0.0389
Epoch 3: val_loss improved from 0.05520 to 0.05376, saving model to models/ann-pot_ativa_inj_barramento11_mse.h5
614/614 [==============================] - 21s 33ms/step - loss: 0.0389 - val_loss: 0.0538
Epoch 4/20
614/614 [==============================] - ETA: 0s - loss: 0.0389
Epoch 4: val_loss did 

In [87]:
cols = ['carga_subrede_138kv',
        'pot_eolica_subrede_138kv',
        'pot_solar_subrede_138kv',
        'tensao_barramento11',
        'tensao_barramento12',
         'tensao_barramento24']

cols_pred = ['pot_ativa_inj_barramento12']
model_id = 'pot_ativa_inj_barramento12_mse'
pot_ativa_inj_barramento12_df = run(df, cols, cols_pred, model_id)

 ========== MODEL pot_ativa_inj_barramento12_mse ============
Epoch 1/20
614/614 [==============================] - ETA: 0s - loss: 0.0539
Epoch 1: val_loss improved from inf to 0.03167, saving model to models/ann-pot_ativa_inj_barramento12_mse.h5
614/614 [==============================] - 23s 36ms/step - loss: 0.0539 - val_loss: 0.0317
Epoch 2/20
614/614 [==============================] - ETA: 0s - loss: 0.0349
Epoch 2: val_loss did not improve from 0.03167
614/614 [==============================] - 19s 31ms/step - loss: 0.0349 - val_loss: 0.0537
Epoch 3/20
613/614 [============================>.] - ETA: 0s - loss: 0.0342
Epoch 3: val_loss did not improve from 0.03167
614/614 [==============================] - 19s 31ms/step - loss: 0.0342 - val_loss: 0.0363
Epoch 4/20
613/614 [============================>.] - ETA: 0s - loss: 0.0288
Epoch 4: val_loss did not improve from 0.03167
614/614 [==============================] - 19s 31ms/step - loss: 0.0288 - val_loss: 0.0407
Epoch 5/20
613/6

In [88]:
cols = ['carga_subrede_138kv',
        'pot_eolica_subrede_138kv',
        'pot_solar_subrede_138kv',
        'tensao_barramento11',
        'tensao_barramento12',
         'tensao_barramento24']

cols_pred = ['pot_ativa_inj_barramento24']
model_id = 'pot_ativa_inj_barramento24_mse'
pot_ativa_inj_barramento24_df = run(df, cols, cols_pred, model_id)

 ========== MODEL pot_ativa_inj_barramento24_mse ============
Epoch 1/20
613/614 [============================>.] - ETA: 0s - loss: 0.0529
Epoch 1: val_loss improved from inf to 0.00543, saving model to models/ann-pot_ativa_inj_barramento24_mse.h5
614/614 [==============================] - 20s 32ms/step - loss: 0.0529 - val_loss: 0.0054
Epoch 2/20
613/614 [============================>.] - ETA: 0s - loss: 0.0212
Epoch 2: val_loss did not improve from 0.00543
614/614 [==============================] - 19s 31ms/step - loss: 0.0212 - val_loss: 0.0111
Epoch 3/20
613/614 [============================>.] - ETA: 0s - loss: 0.0180
Epoch 3: val_loss improved from 0.00543 to 0.00504, saving model to models/ann-pot_ativa_inj_barramento24_mse.h5
614/614 [==============================] - 19s 31ms/step - loss: 0.0180 - val_loss: 0.0050
Epoch 4/20
614/614 [==============================] - ETA: 0s - loss: 0.0141
Epoch 4: val_loss did not improve from 0.00504
614/614 [==============================] 

In [90]:
cols = ['carga_subrede_138kv',
        'pot_eolica_subrede_138kv',
        'pot_solar_subrede_138kv',
        'tensao_barramento11',
        'tensao_barramento12',
         'tensao_barramento24']

cols_pred = ['pot_reativa_inj_barramento11']
model_id = 'pot_reativa_inj_barramento11_mse'
pot_reativa_inj_barramento11_df = run(df, cols, cols_pred, model_id)

 ========== MODEL pot_reativa_inj_barramento11_mse ============
Epoch 1/20
613/614 [============================>.] - ETA: 0s - loss: 0.4454
Epoch 1: val_loss improved from inf to 0.54915, saving model to models/ann-pot_reativa_inj_barramento11_mse.h5
614/614 [==============================] - 20s 31ms/step - loss: 0.4453 - val_loss: 0.5492
Epoch 2/20
613/614 [============================>.] - ETA: 0s - loss: 0.3153
Epoch 2: val_loss did not improve from 0.54915
614/614 [==============================] - 19s 30ms/step - loss: 0.3152 - val_loss: 0.7421
Epoch 3/20
613/614 [============================>.] - ETA: 0s - loss: 0.2679
Epoch 3: val_loss did not improve from 0.54915
614/614 [==============================] - 19s 30ms/step - loss: 0.2679 - val_loss: 0.7050
Epoch 4/20
614/614 [==============================] - ETA: 0s - loss: 0.2261
Epoch 4: val_loss did not improve from 0.54915
614/614 [==============================] - 19s 31ms/step - loss: 0.2261 - val_loss: 1.0870
Epoch 5/20
6

In [92]:
cols = ['carga_subrede_138kv',
        'pot_eolica_subrede_138kv',
        'pot_solar_subrede_138kv',
        'tensao_barramento11',
        'tensao_barramento12',
         'tensao_barramento24']

cols_pred = ['pot_reativa_inj_barramento11']
model_id = 'pot_reativa_inj_barramento11_mae'
pot_reativa_inj_barramento11_mae_df = run(df, cols, cols_pred, model_id)

 ========== MODEL pot_reativa_inj_barramento11_mae ============
Epoch 1/20
613/614 [============================>.] - ETA: 0s - loss: 0.4573
Epoch 1: val_loss improved from inf to 0.60245, saving model to models/ann-pot_reativa_inj_barramento11_mae.h5
614/614 [==============================] - 19s 31ms/step - loss: 0.4572 - val_loss: 0.6024
Epoch 2/20
613/614 [============================>.] - ETA: 0s - loss: 0.4010
Epoch 2: val_loss improved from 0.60245 to 0.56796, saving model to models/ann-pot_reativa_inj_barramento11_mae.h5
614/614 [==============================] - 21s 34ms/step - loss: 0.4010 - val_loss: 0.5680
Epoch 3/20
613/614 [============================>.] - ETA: 0s - loss: 0.3617
Epoch 3: val_loss improved from 0.56796 to 0.56006, saving model to models/ann-pot_reativa_inj_barramento11_mae.h5
614/614 [==============================] - 19s 31ms/step - loss: 0.3618 - val_loss: 0.5601
Epoch 4/20
613/614 [============================>.] - ETA: 0s - loss: 0.3380
Epoch 4: val_l

In [93]:
cols = ['carga_subrede_138kv',
        'pot_eolica_subrede_138kv',
        'pot_solar_subrede_138kv',
        'tensao_barramento11',
        'tensao_barramento12',
         'tensao_barramento24']

cols_pred = ['pot_reativa_inj_barramento12']
model_id = 'pot_reativa_inj_barramento12_mae'
pot_reativa_inj_barramento12_mae_df = run(df, cols, cols_pred, model_id)

 ========== MODEL pot_reativa_inj_barramento12_mae ============
Epoch 1/20
613/614 [============================>.] - ETA: 0s - loss: 0.3652
Epoch 1: val_loss improved from inf to 0.45287, saving model to models/ann-pot_reativa_inj_barramento12_mae.h5
614/614 [==============================] - 19s 30ms/step - loss: 0.3651 - val_loss: 0.4529
Epoch 2/20
613/614 [============================>.] - ETA: 0s - loss: 0.2896
Epoch 2: val_loss did not improve from 0.45287
614/614 [==============================] - 21s 34ms/step - loss: 0.2896 - val_loss: 0.4673
Epoch 3/20
613/614 [============================>.] - ETA: 0s - loss: 0.2620
Epoch 3: val_loss did not improve from 0.45287
614/614 [==============================] - 19s 31ms/step - loss: 0.2619 - val_loss: 0.5063
Epoch 4/20
613/614 [============================>.] - ETA: 0s - loss: 0.2434
Epoch 4: val_loss did not improve from 0.45287
614/614 [==============================] - 19s 31ms/step - loss: 0.2435 - val_loss: 0.4659
Epoch 5/20
6

In [94]:
cols = ['carga_subrede_138kv',
        'pot_eolica_subrede_138kv',
        'pot_solar_subrede_138kv',
        'tensao_barramento11',
        'tensao_barramento12',
         'tensao_barramento24']

cols_pred = ['pot_reativa_inj_barramento24']
model_id = 'pot_reativa_inj_barramento24_mae'
pot_reativa_inj_barramento24_mae_df = run(df, cols, cols_pred, model_id)

 ========== MODEL pot_reativa_inj_barramento24_mae ============
Epoch 1/20
613/614 [============================>.] - ETA: 0s - loss: 0.2738
Epoch 1: val_loss improved from inf to 0.28114, saving model to models/ann-pot_reativa_inj_barramento24_mae.h5
614/614 [==============================] - 22s 35ms/step - loss: 0.2738 - val_loss: 0.2811
Epoch 2/20
614/614 [==============================] - ETA: 0s - loss: 0.2092
Epoch 2: val_loss did not improve from 0.28114
614/614 [==============================] - 21s 34ms/step - loss: 0.2092 - val_loss: 0.2886
Epoch 3/20
613/614 [============================>.] - ETA: 0s - loss: 0.1943
Epoch 3: val_loss did not improve from 0.28114
614/614 [==============================] - 21s 33ms/step - loss: 0.1943 - val_loss: 0.2920
Epoch 4/20
614/614 [==============================] - ETA: 0s - loss: 0.1873
Epoch 4: val_loss did not improve from 0.28114
614/614 [==============================] - 19s 31ms/step - loss: 0.1873 - val_loss: 0.2859
Epoch 5/20
6

In [ ]:
cols = ['carga_subrede_138kv',
        'pot_eolica_subrede_138kv',
        'pot_solar_subrede_138kv',
        'tensao_barramento11',
        'tensao_barramento12',
         'tensao_barramento24']

cols_pred = ['pot_reativa_inj_barramento24']
model_id = 'pot_reativa_inj_barramento24_mse'
pot_reativa_inj_barramento24_mse_df = run(df, cols, cols_pred, model_id)

In [96]:
cols = ['carga_subrede_138kv',
        'pot_eolica_subrede_138kv',
        'pot_solar_subrede_138kv',
        'tensao_barramento11',
        'tensao_barramento12',
         'tensao_barramento24']

cols_pred = ['pot_reativa_inj_barramento24']
model_id = 'pot_reativa_inj_barramento24_mse'
pot_reativa_inj_barramento24_mse_df = run(df, cols, cols_pred, model_id)

 ========== MODEL pot_reativa_inj_barramento24_mse ============
Epoch 1/20
613/614 [============================>.] - ETA: 0s - loss: 0.1551
Epoch 1: val_loss improved from inf to 0.19439, saving model to models/ann-pot_reativa_inj_barramento24_mse.h5
614/614 [==============================] - 20s 31ms/step - loss: 0.1551 - val_loss: 0.1944
Epoch 2/20
613/614 [============================>.] - ETA: 0s - loss: 0.0778
Epoch 2: val_loss did not improve from 0.19439
614/614 [==============================] - 19s 31ms/step - loss: 0.0777 - val_loss: 0.2265
Epoch 3/20
613/614 [============================>.] - ETA: 0s - loss: 0.0635
Epoch 3: val_loss did not improve from 0.19439
614/614 [==============================] - 19s 31ms/step - loss: 0.0635 - val_loss: 0.2237
Epoch 4/20
613/614 [============================>.] - ETA: 0s - loss: 0.0560
Epoch 4: val_loss did not improve from 0.19439
614/614 [==============================] - 19s 31ms/step - loss: 0.0560 - val_loss: 0.2415
Epoch 5/20
6

In [97]:
cols = ['carga_subrede_138kv',
        'pot_eolica_subrede_138kv',
        'pot_solar_subrede_138kv',
        'tensao_barramento11',
        'tensao_barramento12',
         'tensao_barramento24']

cols_pred = ['pot_reativa_inj_barramento12']
model_id = 'pot_reativa_inj_barramento12_mse'
pot_reativa_inj_barramento12_mse_df = run(df, cols, cols_pred, model_id)

 ========== MODEL pot_reativa_inj_barramento12_mse ============
Epoch 1/20
613/614 [============================>.] - ETA: 0s - loss: 0.2578
Epoch 1: val_loss improved from inf to 0.41447, saving model to models/ann-pot_reativa_inj_barramento12_mse.h5
614/614 [==============================] - 20s 33ms/step - loss: 0.2577 - val_loss: 0.4145
Epoch 2/20
613/614 [============================>.] - ETA: 0s - loss: 0.1603
Epoch 2: val_loss did not improve from 0.41447
614/614 [==============================] - 20s 32ms/step - loss: 0.1602 - val_loss: 0.4774
Epoch 3/20
613/614 [============================>.] - ETA: 0s - loss: 0.1413
Epoch 3: val_loss did not improve from 0.41447
614/614 [==============================] - 19s 32ms/step - loss: 0.1413 - val_loss: 0.4777
Epoch 4/20
613/614 [============================>.] - ETA: 0s - loss: 0.1170
Epoch 4: val_loss did not improve from 0.41447
614/614 [==============================] - 19s 31ms/step - loss: 0.1170 - val_loss: 0.5408
Epoch 5/20
6

In [109]:
cols = ['carga_subrede_138kv',
        'pot_eolica_subrede_138kv',
        'pot_solar_subrede_138kv',
        'tensao_barramento11',
        'tensao_barramento12',
         'tensao_barramento24']

cols_pred = ['pot_reativa_inj_barramento11', 'pot_reativa_inj_barramento12', 'pot_reativa_inj_barramento24']
model_id = 'pot_reativa_inj_barramento11_12_24_mse'
pot_reativa_inj_barramento11_12_24_mse_df = run(df, cols, cols_pred, model_id)

 ========== MODEL pot_reativa_inj_barramento11_12_24_mse ============
Epoch 1/20
614/614 [==============================] - ETA: 0s - loss: 0.2817
Epoch 1: val_loss improved from inf to 0.44589, saving model to models/ann-pot_reativa_inj_barramento11_12_24_mse.h5
614/614 [==============================] - 20s 32ms/step - loss: 0.2817 - val_loss: 0.4459
Epoch 2/20
614/614 [==============================] - ETA: 0s - loss: 0.1967
Epoch 2: val_loss did not improve from 0.44589
614/614 [==============================] - 19s 30ms/step - loss: 0.1967 - val_loss: 0.5681
Epoch 3/20
613/614 [============================>.] - ETA: 0s - loss: 0.1623
Epoch 3: val_loss did not improve from 0.44589
614/614 [==============================] - 20s 33ms/step - loss: 0.1623 - val_loss: 0.8244
Epoch 4/20
614/614 [==============================] - ETA: 0s - loss: 0.1481
Epoch 4: val_loss did not improve from 0.44589
614/614 [==============================] - 21s 34ms/step - loss: 0.1481 - val_loss: 1.1066
